In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import os
import time
import regex as re

## Setando Programação Paralela

In [2]:
MAX_WORKERS = os.cpu_count()
print(MAX_WORKERS)

16


## Buscar Links de todos os Produtos

In [3]:
def search_links(categoria, page,product_links):
    url = f'https://www.farmaponte.com.br/{categoria}/?p={page}'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url,headers=headers)
    if response.status_code == 200:
        html_content = response.content
        soup = BeautifulSoup(html_content, 'html.parser')
        links = soup.find_all('a', class_='item-image')

        base_url = "https://www.farmaponte.com.br"

        for link in links:
            href = link.get('href')
            url = f"{base_url}{href}"
            if url not in product_links:
                product_links.append(url)
                print(len(product_links))
                print(page)
        return ''
    else:
        return None

categorias = ['suplementos','saude','beleza','dermocosmeticos','mamae-e-bebe','ortopedicos','cuidados-diarios','conveniencia']
product_links = []

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    for categoria in categorias:
        page = 1
        while True:
            urls = executor.submit(search_links,categoria,page,product_links).result()
            if urls is not None:
                page += 1
                
        # Tentativa com próxima página
            else: 
                page += 1

                urls = executor.submit(search_links,categoria,page,product_links).result()
                if urls is not None:
                    pass
                else:
                    break

1
1
2
1
3
1
4
1
5
1
6
1
7
1
8
1
9
1
10
1
11
1
12
1
13
1
14
1
15
1
16
1
17
1
18
1
19
1
20
1
21
2
22
2
23
2
24
2
25
2
26
2
27
2
28
2
29
2
30
2
31
2
32
2
33
2
34
2
35
2
36
2
37
2
38
2
39
2
40
2
41
3
42
3
43
3
44
3
45
3
46
3
47
3
48
3
49
3
50
3
51
3
52
3
53
3
54
3
55
3
56
3
57
3
58
3
59
3
60
3
61
4
62
4
63
4
64
4
65
4
66
4
67
4
68
4
69
4
70
4
71
4
72
4
73
4
74
4
75
4
76
4
77
4
78
4
79
4
80
4
81
5
82
5
83
5
84
5
85
5
86
5
87
5
88
5
89
5
90
5
91
5
92
5
93
5
94
1
95
1
96
1
97
1
98
1
99
1
100
1
101
1
102
1
103
1
104
1
105
1
106
1
107
1
108
1
109
1
110
1
111
1
112
1
113
1
114
2
115
2
116
2
117
2
118
2
119
2
120
2
121
2
122
2
123
2
124
2
125
2
126
2
127
2
128
2
129
2
130
2
131
2
132
2
133
2
134
3
135
3
136
3
137
3
138
3
139
3
140
3
141
3
142
3
143
3
144
3
145
3
146
3
147
3
148
3
149
3
150
3
151
3
152
3
153
3
154
4
155
4
156
4
157
4
158
4
159
4
160
4
161
4
162
4
163
4
164
4
165
4
166
4
167
4
168
4
169
4
170
4
171
4
172
4
173
4
174
5
175
5
176
5
177
5
178
5
179
5
180
5
181
5
182
5
183
5
184
5
185


In [4]:
product_links

['https://www.farmaponte.com.br/aflexa-max-60cps/p',
 'https://www.farmaponte.com.br/desodalina-600mg-60-capsulas/p',
 'https://www.farmaponte.com.br/suplemento-alimentar-nutren-senior-sem-sabor-lata-1-unidade-com-740g/p',
 'https://www.farmaponte.com.br/ensure-abbott-lata-baunilha-400g/p',
 'https://www.farmaponte.com.br/energisan-4-cpsx20/p',
 'https://www.farmaponte.com.br/nutren-senior-zero-chocolate-1-unidade-com-200ml/p',
 'https://www.farmaponte.com.br/nutren-senior-po-ssab-370gr/p',
 'https://www.farmaponte.com.br/creatina-po-limao-150gr/p',
 'https://www.farmaponte.com.br/nutren-senior-zero-mix-de-frutas-1-unidade-com-200ml/p',
 'https://www.farmaponte.com.br/suplemento-alimentar-nutren-senior-baunilha-lata-1-unidade-com-370g/p',
 'https://www.farmaponte.com.br/ensure-abbott-lata-morango-400g/p',
 'https://www.farmaponte.com.br/nutren-senior-chocolate-740-gramas/p',
 'https://www.farmaponte.com.br/suplemento-alimentar-nutren-senior-cafe-com-leite-lata-1-unidade-com-740g/p',
 '

In [5]:
df = pd.DataFrame({'Links': product_links})
df.to_csv('links.csv')


## Buscando dados em cada item

In [6]:
def search_ean(soup):
    try:
        ean = soup.find('meta', itemprop='gtin13')['content']
        return ean
    except:
        return None

def search_produto(soup):
    try:
        nome_produto = soup.find('h1', class_='name').text.strip()
        return nome_produto
    except:
        return None

def search_brand(soup):
    try:
        marca_tag = soup.find('a', class_='text-primary font-weight-bold')
        marca = marca_tag.text.strip()
        return marca
    except:
        return None
    
def search_farmacia(soup):
    try:
        return 'Farma Ponte'
    except:
        return None
    
def search_precos_desconto(soup):
        try:
            preco_sem_desconto = soup.find('p', class_='unit-price').get_text(strip=True)
            preco_sem_desconto = float(preco_sem_desconto.replace('R$', '').replace(',', '.'))
        except:
            return None,None,None
        try:
            preco_com_desconto = soup.find('p', class_='sale-price').strong.get_text(strip=True)
            preco_com_desconto = float(preco_com_desconto.replace('R$', '').replace(',', '.'))
        except:
            preco_com_desconto = preco_sem_desconto
        try:
            desconto_tag = soup.find('span', class_='discount')
            desconto_texto = desconto_tag.text.strip().replace('% off', '')
            porcentagem_desconto = float(desconto_texto)
        except:
            porcentagem_desconto = 0.0

        return preco_com_desconto,preco_sem_desconto,porcentagem_desconto
    
def search_descricao(soup):
    try:
        descricao_div = soup.find('div', class_='text-body')
        descricao = descricao_div.get_text(strip=True)

        return descricao
    except:
        return None

def buscar_info_products(url,ean,produto,marca,farmacia,preco_com_desconto,preco_sem_desconto,desconto,descricao,i):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url,headers=headers)
    print(i)
    
    if response.status_code == 200:
        html_content = response.content
        soup = BeautifulSoup(html_content, 'html.parser')
        ean.append(search_ean(soup))
        produto.append(search_produto(soup))
        marca.append(search_brand(soup))
        farmacia.append(search_farmacia(soup))
        descricao.append(search_descricao(soup))
        product_preco_desconto,product_preco_sem_desconto,product_desconto = search_precos_desconto(soup)
        preco_com_desconto.append(product_preco_desconto)
        preco_sem_desconto.append(product_preco_sem_desconto)
        desconto.append(product_desconto)
    else:
        ean.append(None)
        produto.append(None)
        marca.append(None)
        farmacia.append(None)
        preco_com_desconto.append(None)
        preco_sem_desconto.append(None)
        desconto.append(None)
        descricao.append(None)



In [7]:
ean = []
produto = []
marca = []
farmacia = []
preco_com_desconto = []
preco_sem_desconto = []
desconto = []
descricao = []
i = 1
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Submeter cada busca em paralelo
    for url in product_links:
        executor.submit(buscar_info_products, url, ean, produto, marca, farmacia, preco_com_desconto, preco_sem_desconto, desconto, descricao,i)
        i += 1

4
2
6
3
5
11
8
10
12
16
15
1
7
13
18
17
9
20
14
2322

24
21
32
3126
19
30

38
29
33
36
37
28
35
34
27
40
39
41
44
45
43
42
49
46
51
5350

48
54
56
55
57
52
47
25
61
62
67
63
70
60
65
72
66
64
68
71
76
58
80
73
75
59
83
69
78
79
8182

77
74
87
85
86
89
88
90
91
92
93
95
84
102
99
101
106
105
97
96
100
94
103
10498

107
119
118
113
114
122
108
115
110
116
109
117
123
111
121
120
124
112
126
125
127
128136

137
138
130
134
135
129
141
131
133
143
139
132
145
151
148
146
153
144
149
155
152
142
156
161
157
154
147
140
158
164
162
159
167
163
166
165
170
169
168
174
176
160
172
150
177
175
182
180
179
181
184
178
187
183
189
192
185
188
186
171
190
191
195
197
196
203
201
202
204
193
198
199
207
205
206
211
173
194
200
214
209
208
210
212
215
217
223
216
220
213
221
222
227
226
229
219
218
231
230
225
233
224
237
234
241
235
243
244
242
238
236
232
239
245
240
246
228
250
248
247
249
253
255
254
251
262
264
256
252
261
263
258
259
269
271
265
272
268
266
274
278
260277

275
280
279
257
270


In [8]:
print(len(ean))
print(len(produto))
print(len(marca))
print(len(farmacia))
print(len(preco_com_desconto))
print(len(preco_sem_desconto))
print(len(desconto))
print(len(descricao))

11556
11556
11556
11556
11556
11556
11556
11556


In [9]:
df = pd.DataFrame({
    'ean': ean,
    'produto': produto,
    'marca': marca,
    'farmacia': farmacia,
    'preco com desconto': preco_com_desconto,
    'preco sem desconto': preco_sem_desconto,
    '% desconto': desconto,
    'descricao': descricao
})

In [10]:
df.head(50)

,ean,produto,marca,farmacia,preco com desconto,preco sem desconto,% desconto,descricao
0,7891158100253,"Ensure Abbott baunilha, lata, 1 unidade com 400g",Ensure,Farma Ponte,84.13,87.55,0.0,O Suplemento Alimentar Ensure Abbott é a compr...
1,7898158498786,"Desodalina 600Mg, 60 Capsulas",Sanibrás,Farma Ponte,29.65,67.36,11.0,Desodalina é um suplemento termogênico à base ...
2,7891000278536,Nutren Senior Zero Lactose Pronto para Beber c...,Nutren,Farma Ponte,17.69,17.69,0.0,O Nutren Senior Zero é ideal para pessoas comm...
3,7891000103487,Complemento Alimentar Nestlé Nutren Senior Zer...,Nutren,Farma Ponte,166.19,166.19,0.0,O Suplemento Alimentar Nutren Senior é indicad...
4,7891105135840,Energisan Kester 4 comprimidos,None,Farma Ponte,8.89,8.89,10.0,Energisan Kester 4 Comprimidos - Energia para ...
5,7891158000119,"Ensure Abbott morango, lata, 1 unidade com 400g",Ensure,Farma Ponte,87.55,87.55,0.0,O Suplemento Alimentar Ensure Abbott é a compr...
6,7908135001363,Creatina em pó Kester Suplement 150g,None,Farma Ponte,69.90,69.90,0.0,Potencialize seu desempenho atlético com a Cre...
7,7891000241547,Complemento Alimentar Nestlé Nutren Senior Zer...,Nutren,Farma Ponte,91.29,91.29,0.0,O Suplemento Alimentar Nutren Senior é indicad...
8,7891000321843,Nutren Senior Chocolate 740 Gramas,Nutren,Farma Ponte,166.19,166.19,0.0,Não Contém Glúten
9,7891105131798,Lissé Hair Solution 60 comprimidos,None,Farma Ponte,69.90,69.90,0.0,"O Nutracêutico Lissé Hair Solution, com 30 cáp..."


In [11]:
df.to_csv('info_produtos_farma_ponte.csv')